In [ ]:
!pip install opencv-python
!pip install mediapipe
!pip install mlcroissant


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------- ----------------------- 4.7/11.6 MB 22.0 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.6 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 26.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/564.9 kB ? eta -:--:--
   --------------------------------------- 564.9/564.9 kB 18.9 MB/s eta 0:00:00
Using cac

In [ ]:
import cv2
import mediapipe as mp

In [ ]:
# Charge les modèles Face Detection de MediaPipe
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [ ]:
# Charge un emoji (PNG avec canal alpha pour la transparence)
emoji = cv2.imread("./images/emoji.png", cv2.IMREAD_UNCHANGED) 

In [ ]:
# Initialise le détecteur de visages
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    image = cv2.imread("./images/photo.png")  # Charge ton image
    if image is None:
        print("Erreur: Image non trouvée.")
    else:
        # Convertit en RGB (MediaPipe a besoin de RGB)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_image)

        if results.detections:
            for detection in results.detections:
                # Récupère la boîte englobante du visage
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = image.shape
                x, y = int(bboxC.xmin * w), int(bboxC.ymin * h)
                width, height = int(bboxC.width * w), int(bboxC.height * h)

                # Redimensionne l'emoji à la taille du visage
                emoji_resized = cv2.resize(emoji, (width, height))

                # Superpose l'emoji (gestion de la transparence)
                for c in range(0, 3):
                    image[y:y+height, x:x+width, c] = \
                        emoji_resized[:, :, c] * (emoji_resized[:, :, 3] / 255.0) + \
                        image[y:y+height, x:x+width, c] * (1.0 - emoji_resized[:, :, 3] / 255.0)
                            # Affiche le résultat
        cv2.imshow("Face avec Emoji", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [ ]:
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

In [ ]:
cap = cv2.VideoCapture(0)
with mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape

                x = max(0, int(bbox.xmin * w))
                y = max(0, int(bbox.ymin * h))
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                
                # Ajuster pour ne pas dépasser l'image
                width = min(width, w - x)
                height = min(height, h - y)

                if width > 0 and height > 0:  # Éviter les dimensions nulles !
                    emoji_resized = cv2.resize(emoji, (width, height))
                    
                    alpha = emoji_resized[:, :, 3] / 255.0
                    for c in range(3):
                        try:
                            frame[y:y+height, x:x+width, c] = (
                                alpha * emoji_resized[:, :, c] + 
                                (1 - alpha) * frame[y:y+height, x:x+width, c]
                            )
                        except ValueError:
                            # En cas d'échec, redimensionner dynamiquement
                            emoji_resized = cv2.resize(emoji_resized, (frame[y:y+height, x:x+width, c].shape[1], frame[y:y+height, x:x+width, c].shape[0]))
                            alpha = emoji_resized[:, :, 3] / 255.0
                            frame[y:y+height, x:x+width, c] = alpha * emoji_resized[:, :, c] + (1 - alpha) * frame[y:y+height, x:x+width, c]

        cv2.imshow('Emoji en Direct', frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
